# Python3 网络爬虫开发实战 | 第 四 章_解析库的使用 (1)

站在巨人的肩膀上学习，感谢崔庆才大神，本系列仅记录学习笔记及个人理解，若有错误，还望批评指教。——ZJ

>原书作者：崔庆才 ，网站 https://cuiqingcai.com/ | https://cuiqingcai.com/5052.html  Github : https://github.com/Germey


---
上一章中，我们实现了一个最基本的爬虫，但提取页面信息时使用的是正则表达式，这还是比较烦琐，而且万一有地方写错了，可能导致匹配失败，所以使用正则表达式提取页面信息多多少少还是有些不方便。

对于网页的节点来说，它可以定义id、class或其他属性。而且节点之间还有层次关系，在网页中可以通过XPath或CSS选择器来定位一个或多个节点。那么，在页面解析时，利用XPath或CSS选择器来提取某个节点，然后再调用相应方法获取它的正文内容或者属性，不就可以提取我们想要的任意信息了吗？

在Python中，怎样实现这个操作呢？不用担心，这种解析库已经非常多，其中比较强大的库有lxml、Beautiful Soup、pyquery等，本章就来介绍这3个解析库的用法。有了它们，我们就不用再为正则表达式发愁，而且解析效率也会大大提高。

XPath，全称XML Path Language，即XML路径语言，它是一门在XML文档中查找信息的语言。它最初是用来搜寻XML文档的，但是它同样适用于HTML文档的搜索。

所以在做爬虫时，我们完全可以使用XPath来做相应的信息抽取。本节中，我们就来介绍XPath的基本用法。

In [2]:
from lxml import etree

text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = etree.tostring(html)
print(result.decode('utf-8'))

<html><body><div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </li></ul>
 </div>
</body></html>


这里首先导入lxml库的etree模块，然后声明了一段HTML文本，调用HTML类进行初始化，这样就成功构造了一个XPath解析对象。这里需要注意的是，HTML文本中的最后一个li节点是没有闭合的，但是etree模块可以自动修正HTML文本。

这里我们调用tostring()方法即可输出修正后的HTML代码，但是结果是bytes类型。这里利用decode()方法将其转成str类型，结果如下：

In [4]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = etree.tostring(html)
print(result.decode('utf-8'))

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">
<html><body><div>&#13;
    <ul>&#13;
         <li class="item-0"><a href="link1.html">first item</a></li>&#13;
         <li class="item-1"><a href="link2.html">second item</a></li>&#13;
         <li class="item-inactive"><a href="link3.html">third item</a></li>&#13;
         <li class="item-1"><a href="link4.html">fourth item</a></li>&#13;
         <li class="item-0"><a href="link5.html">fifth item</a>&#13;
     </li></ul>&#13;
 </div></body></html>


### 5. 所有节点

我们一般会用//开头的XPath规则来选取所有符合要求的节点。这里以前面的HTML文本为例，如果要选取所有节点，可以这样实现：

In [5]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//*')
print(result)

[<Element html at 0x1f3a159f4c8>, <Element body at 0x1f3a1557ac8>, <Element div at 0x1f3a159f248>, <Element ul at 0x1f3a159f288>, <Element li at 0x1f3a159f148>, <Element a at 0x1f3a159f5c8>, <Element li at 0x1f3a159f608>, <Element a at 0x1f3a159f648>, <Element li at 0x1f3a159f688>, <Element a at 0x1f3a159f588>, <Element li at 0x1f3a159f6c8>, <Element a at 0x1f3a159f708>, <Element li at 0x1f3a159f748>, <Element a at 0x1f3a159f788>]


当然，此处匹配也可以指定节点名称。如果想获取所有li节点，示例如下：

In [7]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li')
print(result)
print(result[0])

[<Element li at 0x1f3a159f388>, <Element li at 0x1f3a159f8c8>, <Element li at 0x1f3a159e148>, <Element li at 0x1f3a1593448>, <Element li at 0x1f3a1593548>]
<Element li at 0x1f3a159f388>


### 6. 子节点

我们通过/或//即可查找元素的子节点或子孙节点。假如现在想选择li节点的所有直接a子节点，可以这样实现：

In [9]:
from lxml import etree

html  = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li/a')
print(result)

[<Element a at 0x1f3a1593388>, <Element a at 0x1f3a1593588>, <Element a at 0x1f3a1593fc8>, <Element a at 0x1f3a15938c8>, <Element a at 0x1f3a1593f88>]


In [11]:
from lxml import etree


html  = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//ul//a')
print(result)

[<Element a at 0x1f3a1593cc8>, <Element a at 0x1f3a1593588>, <Element a at 0x1f3a1593248>, <Element a at 0x1f3a15938c8>, <Element a at 0x1f3a15b8188>]


In [12]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//ul/a')
print(result)

[]


### 7. 父节点

我们知道通过连续的/或//可以查找子节点或子孙节点，那么假如我们知道了子节点，怎样来查找父节点呢？这可以用..来实现。

比如，现在首先选中href属性为link4.html的a节点，然后再获取其父节点，然后再获取其class属性，相关代码如下：

from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//a[@href="link4.html"]/../@class')
print(result)

In [14]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//a[@href="link4.html"]/parent::*/@class')
print(result)

['item-1']


### 8. 属性匹配

在选取的时候，我们还可以用@符号进行属性过滤。比如，这里如果要选取class为item-1的li节点，可以这样实现:

In [15]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li[@class="item-0"]')
print(result)


[<Element li at 0x1f3a15575c8>, <Element li at 0x1f3a159ee08>]


### 9. 文本获取

我们用XPath中的text()方法获取节点中的文本，接下来尝试获取前面li节点中的文本，相关代码如下：

In [16]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li[@class="item-0"]/text()')
print(result)

['\r\n     ']


奇怪的是，我们并没有获取到任何文本，只获取到了一个换行符，这是为什么呢？因为XPath中text()前面是/，而此处/的含义是选取直接子节点，很明显li的直接子节点都是a节点，文本都是在a节点内部的，所以这里匹配到的结果就是被修正的li节点内部的换行符，因为自动修正的li节点的尾标签换行了。

即选中的是这两个节点：

```
<li class="item-0"><a href="link1.html">first item</a></li>
<li class="item-0"><a href="link5.html">fifth item</a>
</li>
```
其中一个节点因为自动修正，li节点的尾标签添加的时候换行了，所以提取文本得到的唯一结果就是li节点的尾标签和a节点的尾标签之间的换行符。

因此，如果想获取li节点内部的文本，就有两种方式，一种是先选取a节点再获取文本，另一种就是使用//。接下来，我们来看下二者的区别。

首先，选取到a节点再获取文本，代码如下：

In [17]:
from lxml import etree
 
html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li[@class="item-0"]/a/text()')
print(result)

['first item', 'fifth item']


In [18]:
from lxml import etree
 
html = etree.parse('./test.html', etree.HTMLParser())
result= html.xpath('//li[@class="item-0"]//text()')
print(result)

['first item', 'fifth item', '\r\n     ']


### 10. 属性获取

我们知道用text()可以获取节点内部文本，那么节点属性该怎样获取呢？其实还是用@符号就可以。例如，我们想获取所有li节点下所有a节点的href属性，代码如下：

In [19]:
from lxml import etree
 
html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li/a/@href')
print(result)

['link1.html', 'link2.html', 'link3.html', 'link4.html', 'link5.html']


### 11. 属性多值匹配

有时候，某些节点的某个属性可能有多个值，例如：

In [20]:
from lxml import etree
text = '''
<li class="li li-first"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[@class="li"]/a/text()')
print(result)

[]


In [21]:
# 这时就需要用contains()函数了，代码可以改写如下：

from lxml import etree
text = '''
<li class="li li-first"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[contains(@class, "li")]/a/text()')
print(result)


['first item']


### 12. 多属性匹配

另外，我们可能还遇到一种情况，那就是根据多个属性确定一个节点，这时就需要同时匹配多个属性。此时可以使用运算符and来连接，示例如下：

In [22]:
from lxml import etree
text = '''
<li class="li li-first" name="item"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[contains(@class, "li") and @name="item"]/a/text()')
print(result)

['first item']


### 13. 按序选择

有时候，我们在选择的时候某些属性可能同时匹配了多个节点，但是只想要其中的某个节点，如第二个节点或者最后一个节点，这时该怎么办呢？

这时可以利用中括号传入索引的方法获取特定次序的节点，示例如下：

In [23]:
from lxml import etree
 
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''

html = etree.HTML(text)
result= html.xpath('//li[1]/a/text()')
print(result)
result = html.xpath('//li[last()]/a/text()')
print(result)
result = html.xpath('//li[position()<3]/a/text()')
print(result)
result = html.xpath('//li[last()-2]/a/text()')
print(result)

['first item']
['fifth item']
['first item', 'second item']
['third item']


### 14. 节点轴选择

XPath提供了很多节点轴选择方法，包括获取子元素、兄弟元素、父元素、祖先元素等，示例如下：

In [29]:
from lxml import etree
 
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html"><span>first item</span></a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = html.xpath('//li[1]/ancestor::*')
print(result)
result = html.xpath('//li[1]/ancestor::div')
print(result)
result = html.xpath('//li[1]/attribute::*')
print(result)
result = html.xpath('//li[1]/child::a[@href="link1.html"]')
print(result)

result = html.xpath('//li[1]/descendant::span')
print(result)
result = html.xpath('//li[1]/following::*[2]')
print(result)
result = html.xpath('//li[1]/following-sibling::*')
print(result)

[<Element html at 0x1f3a15cd3c8>, <Element body at 0x1f3a15cd2c8>, <Element div at 0x1f3a15cdfc8>, <Element ul at 0x1f3a15cd208>]
[<Element div at 0x1f3a15cdfc8>]
['item-0']
[<Element a at 0x1f3a15cd7c8>]
[<Element span at 0x1f3a15cd288>]
[<Element a at 0x1f3a15cdfc8>]
[<Element li at 0x1f3a15cd7c8>, <Element li at 0x1f3a15cd2c8>, <Element li at 0x1f3a15cd388>, <Element li at 0x1f3a15cd348>]
